In [1]:
%load_ext autoreload
%autoreload 2

# Mesh Refinement and Convergence

In [2]:
import json
from pathlib import Path

from dolfin import *
import dolfin as df
import numpy as np
import matplotlib.pyplot as plt

import neuralthreesome.Glutamate as gm

In [3]:
def copy_state(state: gm.GlModel.State) -> gm.GlModel.State:
    return gm.GlModel.State(state.g.copy(True), state.m_AMPA.copy(True))

In [4]:
def state_init_zero(V: FunctionSpace, V_interface: FunctionSpace):
    g0 = df.interpolate(Constant(0.0), V)
    m_AMPA0 = df.interpolate(Constant(0.0), V_interface)
    return gm.GlModel.State(g0, m_AMPA0)

In [5]:
class Errs():
    def __init__(self) -> None:
        self.dts = []
        self.errs = []
        self.interface_errs = []

    def add_entry(self, dt: float, err: float, interface_err: float) -> None:
        self.dts.append(dt)
        self.errs.append(err)
        self.interface_errs.append(interface_err)

    def save_json(self, filepath: Path) -> None:
        json_obj = json.dumps(self.__dict__, indent=4)
        with open(str(filepath), "w") as file:
            file.write(json_obj)

    def load_json(self, filepath: Path) -> None:
        json_obj = None
        with open(str(filepath), "r") as file:
            json_obj = json.load(file)
            self.dts = json_obj["dts"]
            self.errs = json_obj["errs"]

In [6]:
def solve_glutamate(mesh_path: Path, output_path: Path, dt, ts=None):
    if ts is None:
        ts = (0.0, 1.0)
        
    subdom_dict = gm.GlModel.subdom_dict
    boundary_dict = gm.GlModel.boundary_dict
    
    gl_mesh = gm.GlMesh()
    gl_mesh.load_xdmf(mesh_path, subdom_dict, boundary_dict)
    
    model = gm.GlModel(gl_mesh)
    solver = gm.Solver(model)
    solver.out_path = output_path
    
    # define stimulus
    stimulus = gm.VesicleMSD(model, ti=[0.1,], offsets=[-0.8,])
    solution, _ = solver.solve(state_init_zero(model.V, model.Vpostsyn), stimulus, ts=ts, dt=dt)
    return solution

In [8]:
# load mesh
mesh_path = Path("../notebooks/synapse_mesh/")

# Errors path
err_obj = Errs()
err_path = Path("../data/Refinements/dt_errs.json")

dt_inf = 1e-2 # asymptotically small time interval as reference 
                # for relative convergence

print("Solving reference solution for rel. convergence: dt={}".format(dt_inf))

output_path = Path("../data/Refinements/dt{:.2}".format(dt_inf))
ref_sol = solve_glutamate(mesh_path, output_path, dt_inf)
dts = [1e-1, 5e-2, 2.5e-2, 1e-2]
for i, dt in enumerate(dts):
    output_path = Path("../data/Refinements/dt{:.2f}".format(dt))
    sol = solve_glutamate(mesh_path, output_path, dt)
    err_g = df.errornorm(ref_sol.g, sol.g, "L2")
    err_m_AMPA = df.errornorm(ref_sol.m_AMPA, sol.m_AMPA, "L2")
    err_obj.add_entry(dt, err_g, err_m_AMPA)
    print("d_g = ", err_g)
    print("d_m_AMPA", err_m_AMPA)

err_obj.save_json(err_path)

Solving reference solution for rel. convergence: dt=0.01
Vpostsyn actually postsyn:  post-synaptic terminal


100%|██████████| 100/100 [00:08<00:00, 12.11it/s]


Vpostsyn actually postsyn:  post-synaptic terminal


100%|██████████| 10/10 [00:00<00:00, 12.15it/s]


d_g =  67548.28444166518
d_m_AMPA inf
Vpostsyn actually postsyn:  post-synaptic terminal


...

d_g =  3830.098270180151
d_m_AMPA inf
Vpostsyn actually postsyn:  post-synaptic terminal


100%|██████████| 40/40 [00:03<00:00, 11.68it/s]


d_g =  3815.9896309775754
d_m_AMPA inf
Vpostsyn actually postsyn:  post-synaptic terminal


100%|██████████| 100/100 [00:08<00:00, 11.63it/s]


d_g =  4568.935686880384
d_m_AMPA inf


In [ ]:
def solve_glutamate_problem()

In [ ]:
## Time Refinement

def glutamate_time_convergence(timesteps, meshfile, geofile):